In [108]:
import numpy as np
import pandas as pd

In [109]:
movies = pd.read_csv('tmdb_5000_movies.csv')

In [110]:
credits = pd.read_csv('tmdb_5000_credits.csv')

In [111]:
movies = movies.merge(credits,on="title")

In [112]:
movies.shape

(4809, 23)

In [113]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [114]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [115]:
movies.dropna(inplace=True)

In [116]:
movies.duplicated().sum()

0

In [117]:
import ast

In [118]:
def convert(obj):
    lis = []
    for i in ast.literal_eval(obj):
        lis.append(i['name'])
    return lis

In [119]:
movies['gen res']= movies['genres'].apply(convert)

In [120]:
movies['keywords']=movies['keywords'].apply(convert)

In [121]:
def getActors(obj):
    lis =[]
    counter =0
    for i in ast.literal_eval(obj):
        if counter!=3:
            lis.append(i['name'])
            counter+=1
        else:
            break
    return lis

In [122]:
movies['cast']= movies['cast'].apply(getActors)

In [123]:
def getDirector(obj):
    lis =[]
    for i in ast.literal_eval(obj):
        if i['job']=="Director":
            lis.append(i['name'])
            break
    return lis

In [124]:
movies['crew'] = movies['crew'].apply(getDirector)

In [125]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [126]:
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [127]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [128]:
new_df = movies[['movie_id','title','tags']]

In [129]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

C:\Users\Dhruv Agrawal\AppData\Local\Temp\ipykernel_11760\1824047427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))


In [130]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

C:\Users\Dhruv Agrawal\AppData\Local\Temp\ipykernel_11760\1380776331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())


In [131]:
import nltk

In [132]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [133]:
def stem(text):
    lis = []
    for i in text.split():
        lis.append(ps.stem(i))
    return " ".join(lis)

In [134]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\Dhruv Agrawal\AppData\Local\Temp\ipykernel_11760\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [148]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=3000, stop_words='english')

In [149]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [150]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [151]:
def recommend(movie):
    movie_index = new_df[new_df['title']==movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse = True, key = lambda x: x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [162]:
recommend('Interstellar')

Guardians of the Galaxy
Silent Running
Moonraker
Space Cowboys
2001: A Space Odyssey


In [163]:
import pickle

In [166]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [167]:
pickle.dump(similarity,open('similarity.pkl','wb'))